In [190]:
import re
import unicodedata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import nltk.sentiment

from wordcloud import WordCloud


plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')


In [286]:
def check_url(url):
    '''This is to check what url's are parsed out of the url that is incoming and returns new_urls 
    and prints out a status code
    '''
    
    user = 'abhisheknaiidu'
    regex = r'''<a\s+(?:[^>]*?\s+)?href="(.*?)"'''
    # Set headers
    headers= {'User-Agent': 'Codeup Data Science'}
    
    # Builds follower url to find the users follower's 
    follower_url = f'https://github.com/{user}?tab=followers'
    # Fetch response from url
    response = get(follower_url, headers=headers)
    
    # Return the user's followers html page
    follower_html = str(BeautifulSoup(response.text, 'html.parser'))
    
    # This regex pulls to pull followers usernames out of the html page
    # set as a set so it doesn't repeat users and remove the first char spot for the '/'
    user_followers = set([r[1][1:] for r in re.findall(r'(link_type\:self"\shref=")(.*?)"', follower_html)])
    
    # Build users repos
    repo_url = f'https://github.com/{user}?tab=repositories'
    
    # get the response from github repos
    response = get(repo_url, headers=headers) 
    
    # Return the user's repo html page
    repo_str = str(BeautifulSoup(response.text, 'html.parser'))
    
    # Filter the first page of repos
    repos = set([r[1] for r in re.findall(
        r'(itemprop="name\scodeRepository">\n)\s*(.*?)<', repo_str)])

    
    return repos
    
    return user_followers
    # Fetches all anchors from page
    
    return anchors
    tmp_urls = list(set(re.findall(regex, str(anchors))))
    readme_urls= list()
    for u in tmp_urls:
        try:
            result = re.findall(r'(http.*?)(blob.*?README\.md)', (str(u)))
            readme_urls.extend([r[0] for r in result])
        except:
            pass
    
    print('STATUS CODE', code)
    return readme_urls

In [287]:
url = 'https://github.com/abhisheknaiidu?tab=followers'
check_url(url)


{'BONGS',
 'Dev-s-Blog',
 'Fusion',
 'a-pop',
 'abhisheknaidu-cli',
 'abhisheknaiidu',
 'abhisheknaiidu.github.io',
 'arm',
 'awesome-github-profile',
 'awesome-github-profile-readme',
 'birthday',
 'cccatalog-frontend',
 'class-karona',
 'creativecommons.github.io-source',
 'dsa',
 'frontend-interview',
 'hacktoberfest2020',
 'iiitdmj-gpa',
 'javascript30',
 'nads',
 'quarantine-days',
 'shortski',
 'social-platform-donut-backend',
 'social-platform-donut-frontend',
 'spin-wheel',
 'startup-backend',
 'startup-frontend',
 'todoist-readme',
 'uPlot',
 'vocabulary'}

In [260]:
repo_str

'\n<!DOCTYPE html>\n\n<html data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">\n<head>\n<meta charset="utf-8"/>\n<link href="https://github.githubassets.com" rel="dns-prefetch"/>\n<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>\n<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>\n<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>\n<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>\n<link href="https://avatars.githubusercontent.com" rel="preconnect"/>\n<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-2f4ea9643ff861e723f03f296396987c.css" integrity="sha512-L06pZD/4Yecj8D8pY5aYfA7oKG6CI8/hlx2K9ZlXOS/j5TnYEjrusaVa9ZIb9O3/tBHmnRFLzaC1ixcafWtaAg==" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-c5cc774753662a380e004d83b021d86e.css" integrity="sha512-xcx3R1NmKjgOAE2DsCHYbu

In [285]:
re.findall(r'(itemprop="name\scodeRepository">\n)\s*(.*?)<', repo_str)
repos = set([r[1] for r in re.findall(r'(itemprop="name\scodeRepository">\n)\s*(.*?)<', repo_str)])
repos

{'BONGS',
 'Dev-s-Blog',
 'Fusion',
 'a-pop',
 'abhisheknaidu-cli',
 'abhisheknaiidu',
 'abhisheknaiidu.github.io',
 'arm',
 'awesome-github-profile',
 'awesome-github-profile-readme',
 'birthday',
 'cccatalog-frontend',
 'class-karona',
 'creativecommons.github.io-source',
 'dsa',
 'frontend-interview',
 'hacktoberfest2020',
 'iiitdmj-gpa',
 'javascript30',
 'nads',
 'quarantine-days',
 'shortski',
 'social-platform-donut-backend',
 'social-platform-donut-frontend',
 'spin-wheel',
 'startup-backend',
 'startup-frontend',
 'todoist-readme',
 'uPlot',
 'vocabulary'}

In [242]:
'name codeRepository'
repos = repo_str.find_all(itemprop='name codeRepository')
repos

[<a href="/abhisheknaiidu/abhisheknaiidu" itemprop="name codeRepository">
         abhisheknaiidu</a>,
 <a href="/abhisheknaiidu/awesome-github-profile-readme" itemprop="name codeRepository">
         awesome-github-profile-readme</a>,
 <a href="/abhisheknaiidu/javascript30" itemprop="name codeRepository">
         javascript30</a>,
 <a href="/abhisheknaiidu/class-karona" itemprop="name codeRepository">
         class-karona</a>,
 <a href="/abhisheknaiidu/BONGS" itemprop="name codeRepository">
         BONGS</a>,
 <a href="/abhisheknaiidu/shortski" itemprop="name codeRepository">
         shortski</a>,
 <a href="/abhisheknaiidu/uPlot" itemprop="name codeRepository">
         uPlot</a>,
 <a href="/abhisheknaiidu/Dev-s-Blog" itemprop="name codeRepository">
         Dev-s-Blog</a>,
 <a href="/abhisheknaiidu/frontend-interview" itemprop="name codeRepository">
         frontend-interview</a>,
 <a href="/abhisheknaiidu/dsa" itemprop="name codeRepository">
         dsa</a>,
 <a href="/abhishe

In [237]:
regex_users = r'(link_type\:self"\shref=")(.*?)"'
users = re.findall(regex_html, str(html))
users

[]

In [21]:
url2 = '/url?q=https://github.com/go-playground/pool/blob/v3/README.md&amp;sa=U&amp;ved=2ahUKEwiW3-OBn_HzAhUSa80KHbVLCMsQFnoECAQQAg&amp;usg=AOvVaw02gP7ocWEC-Vrb9qnOae2X'